In [40]:
import numpy as np 
import re

# corpus=3
# term_incidence_matrix = np.array([[2,0,1],[0,1,2],[1,1,0],[0,1,0],[1,0,0],[1,1,1],[0,1,1],[1,2,1],[2,0,2],[1,2,0],[0,3,1]],dtype=np.float32)   
# query=np.array([[1],[0],[0],[0],[0],[1],[0],[0],[0],[1],[0]],dtype=np.float32)

no_files_in_corpus = 3
term_incidence_matrix,term_collection=build_corpus(no_files_in_corpus)

query = "brown fox jump"
token = tokenize(query)

query = [0 for i in range(len(term_collection))]
for t in token:
    index = term_collection.get(t, None)

    if index is not None:
        query[index] = query[index] + 1

'''
Rebuild the query
'''
query = np.array(query)
print("Query is:")
print(query)

print("Original :") 
print(term_incidence_matrix) 

# Compute the factor by Singular Value 
# Decomposition 
np.set_printoptions(precision=5,suppress=True)
U, S, Vt = np.linalg.svd(term_incidence_matrix, full_matrices=False) 
S=np.diag(S)
V=np.transpose(Vt)

# Print the result 
print("\nFactorS of the given term_incidence_matrix by Singular Value Decomposition are:") 
print("\nU Matrix=", U, "\n\nS Matrix=", S, "\n\nV_transpose_matrix=", Vt, "\n\nV_matrix=", V) 

qt=np.transpose(query)
print("\nq=", query, "\n\nqt=", qt) 

k=2
Uk=U[:,:k]
Vk=V[:,:k]
Sk=S[:k,:k]
Ski=np.linalg.inv(Sk) 


doc=[[] for i in range(no_files_in_corpus)]
for i in range(0,no_files_in_corpus):
    doc[i]=Vk[i]

print("\nUk=", Uk, "\n\nVk=", Vk, "\n\nsk=", Sk) 
qnew=np.dot((np.dot(qt,Uk)),Ski)
print("\nqnew=", qnew)


magqnew=np.linalg.norm(qnew)
sim=np.zeros(no_files_in_corpus)

for i in range(0,no_files_in_corpus):
    sim[i]=(np.dot(qnew,doc[i])/(magqnew*np.linalg.norm(doc[i])))

print(sim)
doc_index = np.argsort(sim)[::-1][:no_files_in_corpus]
print("document ranks are:")
print(doc_index) 



Processing the documents in corpus (Collecting all terms) . . . Done
Query is:
[0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0]
Original :
[[0, 0, 1], [1, 0, 0], [1, 0, 1], [0, 1, 1], [1, 0, 0], [1, 1, 1], [1, 0, 0], [1, 0, 0], [0, 1, 1], [1, 1, 1], [0, 1, 0], [0, 1, 0], [1, 1, 0], [1, 1, 0], [1, 0, 1], [0, 0, 1]]

FactorS of the given term_incidence_matrix by Singular Value Decomposition are:

U Matrix= [[-0.1324  -0.19821 -0.35355]
 [-0.15704  0.33422 -0.     ]
 [-0.28944  0.136   -0.35355]
 [-0.2648  -0.39643 -0.     ]
 [-0.15704  0.33422 -0.     ]
 [-0.42184 -0.06221  0.     ]
 [-0.15704  0.33422 -0.     ]
 [-0.15704  0.33422 -0.     ]
 [-0.2648  -0.39643 -0.     ]
 [-0.42184 -0.06221  0.     ]
 [-0.1324  -0.19821  0.35355]
 [-0.1324  -0.19821  0.35355]
 [-0.28944  0.136    0.35355]
 [-0.28944  0.136    0.35355]
 [-0.28944  0.136   -0.35355]
 [-0.1324  -0.19821 -0.35355]] 

S Matrix= [[4.09201 0.      0.     ]
 [0.      2.29247 0.     ]
 [0.      0.      2.     ]] 

V_transpose_matrix= [[-0.6426

In [37]:
def tokenize(text, term_collection = None):
    '''
    Splitting the text aginst some pre-defined delimiter
    '''
    '''Tokenization'''
    split_chars = [" ", "\n", "\-", ",", "\.", "\"", "\(", "\)", "\?", "\[", "\]", "\*", ";", "\\", "/"]
    delimiter = '|'.join(split_chars)
    tokens = re.split(delimiter, text)

    tokens = list(map(lambda x: x.lower(), tokens))

    if term_collection is not None:
        for t in tokens:
            term_collection.append(t)

        return tokens

    else:
        return tokens

    '''END'''


def build_corpus(no_files_in_corpus):
    '''
    Building corpus as term_collection
    '''
    term_collection, all_tokens = [], []

    '''
    Collecting all terms present in the collection
    '''
    print("\nProcessing the documents in corpus (Collecting all terms) . . . ", end = '')
    for i in range(1,no_files_in_corpus+1):
        with open('doc' + str(i) + '.txt', 'r') as f:
            text = f.read()

            '''Storing document-wise and in-total at one time'''
            all_tokens.append(tokenize(text, term_collection))

    # Sorting the terms lexicographically and make it a dictionary
    term_collection = sorted(list(set(term_collection)))
    term_collection = {term : index for index, term in enumerate(term_collection)}

    '''
    Building term-document incedence matrix
    '''
    corpus = [[0 for j in range(no_files_in_corpus)] for i in range(len(term_collection))]

    for doc_index, doc_tokens in enumerate(all_tokens):
        for token in doc_tokens:
            row_index, col_index = term_collection[token], doc_index
            corpus[row_index][col_index] = corpus[row_index][col_index] + 1

    '''
    END
    '''
    print("Done")

    return corpus, term_collection
